In [1]:
from tqdm import tqdm
import pandas as pd 
import os
from snorkel.labeling import PandasLFApplier,LFAnalysis,LabelingFunction
from snorkel.labeling.model.label_model import LabelModel
from snorkel.labeling.apply.dask import PandasParallelLFApplier
import re
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
import pytrends
from pytrends.request import TrendReq
import time
import datetime
from datetime import datetime, date, time
from nltk import ngrams

In [ ]:
#path information
task='task1'# or 'task2' # specify task
root_path='/repo1/code/autoreview/'
data_path=root_path+'data/'+task+'/'#path to save retrieved articles abstract
keyword_path=data_path+'keywords/'#keyword list
save_path=root_path+'results/'+task+'/'
sentence_file_name='hypercoagulable.pkl'
sentence_embedding_file_name='sentence_embedding.pkl'

In [2]:
def load_keyword(keyword_file_name):
    """
    Read keyword list
    
    input:
        keyword_type_name: str, file name, e.g., `keylist.txt`
    output:
        list of string, e.g., ['keyword1', 'keyword2', 'keyword3']
        
    """
    with open(keyword_file_name, "r") as f:
        keylist=f.read().split(',')
    return keylist

In [50]:
def loop_labing(keywordslist,viruslist,triallist,mustlist,maxngramnu):
    
    # keywordslist is the concept we want
    # viruslist is what type of virus we want to loop up
    # triallist is concept of trial we want to loop up
    # mustlist is concept of  we want the high ranking sentence should have
    # maxngramnu is the maxinum number of word window
    
    # lookup keywords in n-words window (for controling the window, we set up the max number of word window)
    def keyword_lookup(x,keywords,maxngram,label):
        threshold=0
        for i in range(maxngram):
            gramab=ngrams(x.sentence.lower().split(), i+1)
            for word in gramab:
                wordlist=list(word)
                if any(word1 in wordlist  for word1 in keywords):
                    threshold=threshold+1
        if threshold>0:
            return label
        return Norelevent

    def make_keyword_lf(keywords,maxngram,name,label=None):
        return LabelingFunction(
            name=f"keyword_{name}",
            f=keyword_lookup,
            resources=dict(keywords=keywords,maxngram=maxngram,label=label),)
    
    # lookup both group keywords in n-words window (for controling the window, we set up the max number of word window)
    
    def keyword_lookup_b(x,keywords1,keywords2,maxngram,label):
        threshold=0
        for i in range(maxngram):
            gramab=ngrams(x.sentence.lower().split(), i+1)
            for word in gramab:
                wordlist=list(word)
                if any(word1 in wordlist  for word1 in keywords1) and any(word1 in wordlist  for word1 in keywords2,):
                    threshold=threshold+1
        if threshold>0:
            return label
        return Norelevent

    def make_keyword_b_lf(keywords1,keywords2,maxngram,name,label=None):
        return LabelingFunction(
            name=f"keyword_b_{name}",
            f=keyword_lookup_b,
            resources=dict(keywords1=keywords1,keywords2=keywords2,maxngram=maxngram,label=label),)
   
    # lookup a group keywords presenting and not presenting in another group of keywords
    
    def not_lookup_b(x,keywords1,keywords2,maxngram,label):
        threshold=0
        for i in range(maxngram):
            gramab=ngrams(x.sentence.lower().split(), i+1)
            for word in gramab:
                wordlist=list(word)
                if any(word1 in wordlist  for word1 in keywords2):
                    threshold=threshold+1
        threshold1=0
        for i in range(maxngram):
            gramab=ngrams(x.sentence.lower().split(), i+1)
            for word in gramab:
                wordlist=list(word)
                if any(word1 in wordlist  for word1 in keywords1):
                    threshold1=threshold1+1
        if threshold<1 and threshold1>0:
            return label
        return Norelevent

    
    def make_not_b_lf(keywords1,keywords2,name,maxngram,label=None):
        return LabelingFunction(
            name=f"not_b_{name}",
            f=not_lookup_b,
            resources=dict(keywords1=keywords1,keywords2=keywords2,maxngram=maxngram,label=label),)   
    
    
      # lookup up number in sentence 
    
    
    def number_lookup(x,keywords,label):
        threshold=0
        sentlist=x.sentence.split()
        trialindex=[i for i, e in enumerate(sentlist) if e in keywords]
        nuindex=[i for i, e in enumerate(sentlist) if str(e).isdigit()]
        for i in trialindex:
            for j in nuindex:
                if i<j:
                    threshold+=1
        if threshold>0:
            return label
        return Norelevent

    def make_number_lf(keywords,label=None):
        return LabelingFunction(
            name=f"number",
            f=number_lookup,
            resources=dict(keywords=keywords,label=label),)

   
    
    Norelevent = -1
    
    keywordfu=make_keyword_lf(keywords=keywordslist,maxngram=maxngramnu,name='keyword',label=1)    
    virusfu=make_keyword_lf(keywords=viruslist,maxngram=maxngramnu,name='virus',label=1)
    keywordfub=make_keyword_b_lf(keywords1=keywordslist,keywords2=mustlist,maxngram=maxngramnu,name='keyword',label=1)    
    virusfub=make_keyword_b_lf(keywords1=viruslist,keywords2=mustlist,maxngram=maxngramnu,name='virus',label=1)
    trialfu=make_keyword_lf(keywords=triallist,name='trial',maxngram=maxngramnu,label=1)
    notfu1=make_not_b_lf(keywords1=mustlist,keywords2=keywordslist,maxngram=maxngramnu,name='notkeyword',label=0)
    notfu2=make_not_b_lf(keywords1=keywordslist,keywords2=triallist,maxngram=maxngramnu,name='nottrial',label=0)
    numberfu=make_number_lf(keywords=triallist,label=1)
    
    allweaklabf=[]

    allweaklabf.append(keywordfu)
    allweaklabf.append(virusfu)
    allweaklabf.append(keywordfub)
    allweaklabf.append(virusfub)
    allweaklabf.append(trialfu)
    allweaklabf.append(notfu1)
    allweaklabf.append(notfu2)
    allweaklabf.append(numberfu)

    
    return allweaklabf


In [51]:
def predict_prob(sentences,lfs,cardinalitynu):
    """
    Predict probability (label) by applying label functions lfs
    Refined from `snorkel_process` 
    
    input:
        lfs: list of snorkel.LabelingFunction
    output:
        sentences_labeled: pd.Dataframe,  [pid, sid, sent, label, prob ]
    """
    
    applier=PandasLFApplier(lfs=lfs)
    applied=applier.apply(df=sentences)
    print(LFAnalysis(L=applied, lfs=lfs).lf_summary())
    
    label_model = LabelModel(cardinality=cardinalitynu,verbose=True)
    label_model.fit(applied)
    sentences['label']=label_model.predict(applied)
    sentences['prob']=label_model.predict_proba(applied)[:,1]
    
    return sentences

In [52]:
#abstract
gold=pd.read_csv('hypercoagulable_sentences.csv')
gold=gold.dropna()
ab=[]
ncord=[]
for i in tqdm(gold.ncord_uid.drop_duplicates()):
    temp=gold[gold.ncord_uid==i]
    tempsent=temp.sentence.values
    tempncord=temp.ncord_uid.drop_duplicates().values[0]
    init=''
    for j in tempsent:
        init=init+j
    ab.append(init)
    ncord.append(tempncord)
goldab=pd.DataFrame(ncord,columns=['ncord_uid'])
goldab['sentence']=ab
goldab=goldab.drop_duplicates()

100%|██████████| 8766/8766 [00:12<00:00, 686.55it/s]


In [53]:

cardinalitynu=2
allweaklabf=loop_labing(keywordslist,viruslist,triallist,mustlist,3)
sentence=predict_prob(goldab,allweaklabf,cardinalitynu)

100%|██████████| 8766/8766 [02:43<00:00, 53.70it/s] 


                   j Polarity  Coverage  Overlaps  Conflicts
keyword_keyword    0      [1]  0.247205  0.247205   0.133356
keyword_virus      1      [1]  0.060575  0.050650   0.006160
keyword_b_keyword  2       []  0.000000  0.000000   0.000000
keyword_b_virus    3      [1]  0.000570  0.000570   0.000570
keyword_trial      4      [1]  0.628109  0.423340   0.000684
not_b_notkeyword   5      [0]  0.001141  0.000799   0.000799
not_b_nottrial     6      [0]  0.133356  0.133356   0.133356
number             7      [1]  0.354438  0.354438   0.000228


In [54]:
sentence=sentence.sort_values(by='prob',ascending=False)
sentence.to_pickle('round_2_ab.pkl')

In [57]:
gold=pd.read_csv('hypercoagulable_sentences.tsv',sep='\t')
gold['sentence']=gold.sentence.astype(str)
cardinalitynu=2
allweaklabf=loop_labing(keywordslist,viruslist,triallist,mustlist,3)
sentence=predict_prob(gold,allweaklabf,cardinalitynu)

100%|██████████| 142901/142901 [04:09<00:00, 572.42it/s] 


                   j Polarity  Coverage  Overlaps  Conflicts
keyword_keyword    0      [1]  0.018810  0.018810   0.015997
keyword_virus      1      [1]  0.015136  0.003261   0.001274
keyword_b_keyword  2       []  0.000000  0.000000   0.000000
keyword_b_virus    3      [1]  0.000035  0.000035   0.000035
keyword_trial      4      [1]  0.078222  0.013121   0.000021
not_b_notkeyword   5      [0]  0.000203  0.000140   0.000140
not_b_nottrial     6      [0]  0.015997  0.015997   0.015997
number             7      [1]  0.009251  0.009251   0.000000


In [58]:
sentence=sentence.sort_values(by='prob',ascending=False)
sentence.to_pickle('round_2_sent.pkl')

In [ ]:
cardinalitynu=2
allweaklabf=loop_labing(keywordslist,viruslist,triallist,mustlist,3)
applier = PandasLFApplier(lfs=allweaklabf)
all_train_l = applier.apply(df=gold)
#applier = PandasParallelLFApplier(lfs=allweaklabf)
#all_train_l = applier.apply(df=allframe1,n_parallel=40)
report=LFAnalysis(L=all_train_l, lfs=allweaklabf).lf_summary()
print(report)
label_model = LabelModel(cardinality=cardinalitynu,verbose=True)
label_model.fit(all_train_l)
predt=label_model.predict(all_train_l)

In [ ]:
gold['label']=predt
prob=label_model.predict_proba(all_train_l)[:,1]
gold['prob_snokel']=prob
gold1=gold[gold.label==1]
gold1=gold1.drop_duplicates()
gold1=gold1.sort_values(by='prob_snokel',ascending=False)

In [ ]:
gold.to_pickle('./round_2.pkl')